# Motor Collisions and Traffic Volumes Data Pipeline
This file takes you through the pipeline the motor collision and traffic volume data goes through in order to ultimately create a master collision-volume dataframe from which the main analysis of this project will be conducted. 

A summary of the files and databases used in this analysis is provided below:

**Database #1: Motor Vehicle Collisions involving Killed or Seriously Injured Persons**

*File 1: Motor Vehicle Collisions with KSI Data (GeoJSON)*
This dataset includes all traffic collisions events where a person was either Killed or Seriously Injured (KSI) from 2006 – 2021. Each collision in this dataset includes information on the date, time, location, cause and circumstances surrounding the collision. 

**Database #2: Traffic Volumes at Intersections for All Modes**
    
*File 2: locations (CSV)*

This dataset contains the coordinates and other geospatial data about each intersection the database has volume counts for.


*File 3: raw-data-2000-2009 (CSV)*

*File 4: raw-data-2010-2019 (CSV)*

*File 5: raw-data-2020-2029 (CSV)*

These datasets contains the turning movement counts observed at specific intersections at specific dates and times, segmented by years 2000-2009, 2010-2019, and 2020-2029. Included are the total volumes observed at specific intersections, segmented by direction of approach, turning movement (if applicable), and mode (car, truck, bus, pedestrian, cyclist, other).


# 1: Downloading the Data

The following section looks to download and convert the various files (containing data on motor collisions, traffic counts, and intersection information) required for this study into dataframes for data cleaning and analysis. 

In [371]:
# Import libraries
import os
import json
import pandas as pd
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import requests
import geopandas as gpd
import folium
from folium import Marker
from geopy.distance import geodesic
from scipy.spatial import KDTree
from scipy.spatial import cKDTree
import contextily as ctx
import warnings
import timeit
from tqdm import tqdm
import time
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
from folium.plugins import MarkerCluster
from folium.map import FeatureGroup
from folium.plugins import FastMarkerCluster

from shapely.ops import unary_union



# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### 1.1 Downloading Motor Collision Data
Pulling the motor collision data from the City of Toronto Open Data Portal and converting the json file into a pandas dataframe.

In [251]:
# Following code pulls dictionary (.json file) on motor collisions
motor_collisions = (requests.
                    get('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/0b6d3a00-7de1-440b-b47c-7252fd13929f/resource/355d4464-eb3c-4780-af79-43dd533ae906/download/Motor%20Vehicle%20Collisions%20with%20KSI%20Data.geojson').
                    json()
                   )

In [252]:
# Initialize an empty list to store flattened dictionaries
flat_motor_collisions = []

# Iterate through the 'features' list in the JSON motor_collisions
for feature in motor_collisions['features']:
    # properties entail the characteristics of the crash
    properties = feature['properties']
    # geometry entails the location
    geometry = feature['geometry']
    flat_properties = properties.copy()
    flat_properties.update(geometry)
    flat_motor_collisions.append(flat_properties)
    
# Create a motor_collisions DataFrame from the flattened motor_collisions
motor_collisions_df = pd.DataFrame(flat_motor_collisions)

In [253]:
motor_collisions_df.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,type,coordinates
0,1,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,None,None,None,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]"
1,2,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,None,None,None,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]"
2,3,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,Yes,None,None,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]"
3,4,893184,2006,2006-01-01,236,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,...,None,Yes,None,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,MultiPoint,"[[-79.318797, 43.699595]]"
4,5,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,Yes,None,None,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]"


### 1.2 Downloading Traffic and Intersection data
Pulling the traffic count and intersection data from the City of Toronto Open Data Portal and converts the csv files into pandas dataframes.

In [254]:
# Data that includes strictly geospatial information on the intersections where counts are taken from
intersections = pd.read_csv('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/traffic-volumes-at-intersections-for-all-modes/resource/7f1243fb-8196-459b-adb8-d9331e1d4b65/download/locations.csv')

# Data on historical intersection volumes from 2000 to 2009
vol_00to09 = pd.read_csv('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/traffic-volumes-at-intersections-for-all-modes/resource/bb7554d9-cd5d-4fad-aa5b-97339a9018df/download/raw-data-2000-2009.csv')

# Data on historical intersection volumes from 2010 to 2019
vol_10to19 = pd.read_csv('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/traffic-volumes-at-intersections-for-all-modes/resource/1f60c668-bb8e-4e1e-ac72-3c6558a03fea/download/raw-data-2010-2019.csv')

# Data on historical intersection volumes from 2020 to 2022
vol_20to29 = pd.read_csv('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/traffic-volumes-at-intersections-for-all-modes/resource/71f08804-46ce-4a92-9e8f-9b0e67927ca6/download/raw-data-2020-2029.csv')

In [255]:
intersections.head()

,_id,location_id,location,lng,lat,centreline_type,centreline_id,px,latest_count_date
0,1,1146,ELLESMERE RD AT PARKINGTON CRES (PX 2296),-79.246253,43.773318,2.0,13446642.0,2296.0,2016-04-05
1,2,1981,YORK MILLS RD E/B & W/B TO DON VALLEY PKWY N/B,-79.334658,43.757336,1.0,440171.0,NaN,2004-04-08
2,3,3468,PARKSIDE DR N/B S OF SPRING RD,-79.454442,43.640512,1.0,30010748.0,NaN,2021-12-15
3,4,3925,RIPLEY AVE AT SOUTH KINGSWAY,-79.475274,43.636780,2.0,13468657.0,NaN,2022-01-27
4,5,3926,BLOOR ST AT SOUTH KINGSWAY & RIVERVIEW GARDENS...,-79.485752,43.648312,2.0,13467247.0,334.0,2022-05-10


In [256]:
vol_00to09.head()

,_id,count_id,count_date,location_id,location,lng,lat,centreline_type,centreline_id,px,...,ex_peds,wx_peds,nx_bike,sx_bike,ex_bike,wx_bike,nx_other,sx_other,ex_other,wx_other
0,1,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,12.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,9.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,10.0,4.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [257]:
vol_00to09.head()

,_id,count_id,count_date,location_id,location,lng,lat,centreline_type,centreline_id,px,...,ex_peds,wx_peds,nx_bike,sx_bike,ex_bike,wx_bike,nx_other,sx_other,ex_other,wx_other
0,1,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,12.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,9.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,10.0,4.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [258]:
vol_00to09.head()

,_id,count_id,count_date,location_id,location,lng,lat,centreline_type,centreline_id,px,...,ex_peds,wx_peds,nx_bike,sx_bike,ex_bike,wx_bike,nx_other,sx_other,ex_other,wx_other
0,1,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,12.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,9.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,8180,2000-01-18,4126,EGLINTON AVE AT PHARMACY AVE (PX 452),-79.297515,43.725651,2.0,13453978.0,452.0,...,10.0,4.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


### 1.3: Downloading Intersection Attributes 

In [259]:
inter_att = (requests.get('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/2c83f641-7808-49ba-b80f-7011851d4e27/resource/8e825e33-d7e1-4e59-b247-5868bf7d66a9/download/Centreline%20Intersection.geojson').
            json()
                   )
# Initialize an empty list to store flattened dictionaries
inter_att_df = []

# Iterate through the 'features' list in the JSON intersection attributes data
for feature in inter_att['features']:
    # properties entail the characteristics of the crash
    properties = feature['properties']
    # geometry entails the location
    geometry = feature['geometry']
    flat_properties = properties.copy()
    flat_properties.update(geometry)
    inter_att_df.append(flat_properties)
    
# Create a motor_collisions DataFrame from the flattened inter_att_df
inter_att_df = pd.DataFrame(inter_att_df)
inter_att_df.head()

,_id,INTERSECTION_ID,DATE_EFFECTIVE,DATE_EXPIRY,ELEVATION_ID,INTERSECTION_DESC,CLASSIFICATION,CLASSIFICATION_DESC,NUMBER_OF_ELEVATIONS,ELEVATION_FEATURE_CODE,...,ELEVATION,ELEVATION_UNIT,HEIGHT_RESTRICTION,HEIGHT_RESTRICTION_UNIT,STATE,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,type,coordinates
0,1,13470264,,,13,Robindale Ave / Rimilton Ave,MNRSL,Minor-Single Level,1,501300,...,0.0,,0.0,,8,200000,-1,1,MultiPoint,"[[-79.5310702158097, 43.6072425849711]]"
1,2,13470193,,,4718,Bellman Ave / Valermo Dr,MNRSL,Minor-Single Level,1,501300,...,0.0,,0.0,,8,200000,-1,4,MultiPoint,"[[-79.5313732423075, 43.609600012102]]"
2,3,13470188,,,32728,Rimilton Ave / Valermo Dr,SEUSL,Pseudo Intersection-Single Level,1,509200,...,0.0,,0.0,,8,200000,-1,5,MultiPoint,"[[-79.5301175801351, 43.6098292200395]]"
3,4,13470203,,,21669,Valermo Dr / Goa Crt,MNRSL,Minor-Single Level,1,501300,...,0.0,,0.0,,8,200000,-1,7,MultiPoint,"[[-79.5331747278101, 43.6091899836547]]"
4,5,13470228,,,36820,Valermo Dr / Thirtieth St,MNRSL,Minor-Single Level,1,501300,...,0.0,,0.0,,8,200000,-1,9,MultiPoint,"[[-79.5355925204638, 43.6086391702897]]"


### 1.4: Downloading Intersection Safety Attributes

In [260]:
traffic_calming_gdf = gpd.read_file('Traffic Calming Database - 04.04.2023V2.shp')
traffic_calming_gdf.head()

,calm_id,street,intersecti,intersec_1,spd_hump,traf_islan,spd_cush,Installed,geometry
0,1.0,Oakridge Drive,Brimley Rd,McCowan Rd,8.0,0.0,0.0,2004,"LINESTRING (325456.886 4842936.976, 325501.009..."
1,3.0,Atlas Avenue,Gloucester Grove,Eglinton Avenue West,2.0,0.0,0.0,1999,"LINESTRING (309818.658 4839638.193, 309785.082..."
2,4.0,Balliol Street,Mt. Pleasant Road,Cleveland Street,2.0,0.0,0.0,1998,"LINESTRING (313963.651 4839945.340, 314325.237..."
3,5.0,Balmoral Avenue,Avenue Road,Yonge Street,6.0,0.0,0.0,1998,"LINESTRING (313413.250 4838443.321, 313363.575..."
4,6.0,Bartlett Avenue North,Geary Avenue,Davenport Road,6.0,0.0,0.0,1999,"LINESTRING (309897.291 4836672.254, 309882.452..."


In [261]:
# Load GeoJSON files into GeoDataFrames
speed_enforcement_gdf = gpd.read_file('Automated Speed Enforcement Locations.geojson')
speed_enforcement_gdf.head()

,_id,Location_Code,Ward,Status,location,FID,geometry
0,1,A007,4 - Parkdale-High Park,Active,Jameson Ave. South of Laxton Ave.,1,MULTIPOINT (-79.45528 43.64253)
1,2,A016,8 - Eglinton-Lawrence,Active,Ridge Hill Dr. West of Old Park Rd.,2,MULTIPOINT (-79.43294 43.70212)
2,3,A017,9 - Davenport,Active,Caledonia Rd. North of Rogers Rd.,3,MULTIPOINT (-79.45843 43.68403)
3,4,A018,9 - Davenport,Active,Gladstone Ave. South of Cross St.,4,MULTIPOINT (-79.42869 43.64658)
4,5,A037,19 - Beaches-East York,Active,Main St. South of Swanwick Ave.,5,MULTIPOINT (-79.29926 43.68172)


In [262]:
red_light_cameras_gdf = gpd.read_file('Red Light Cameras Data.geojson')

# Convert the Timestamp column to string in red_light_cameras_gdf
red_light_cameras_gdf['ACTIVATION_DATE'] = red_light_cameras_gdf['ACTIVATION_DATE'].astype(str)
red_light_cameras_gdf.head()

,_id,INTERSECTION_ID,LINEAR_NAME_FULL_1,LINEAR_NAME_FULL_2,MI_PRINX,OBJECTID,ID,CLIENT_STREET_1,CLIENT_STREET_2,RLC,...,DISTRICT,WARD_1,WARD_2,WARD_3,WARD_4,POLICE_DIVISION_1,POLICE_DIVISION_2,POLICE_DIVISION_3,ACTIVATION_DATE,geometry
0,1,13465959,Richmond St E,Parliament St,1,1,1,RICHMOND ST E,PARLIAMENT ST,6001,...,Toronto and East York,Toronto Centre(13),,,,51,,,2007-11-09 05:00:00,MULTIPOINT (-79.36402 43.65456)
1,2,13467993,Lake Shore Blvd W,York St,2,2,2,LAKE SHORE BLVD W,YORK ST,6002,...,Toronto and East York,Spadina-Fort York(10),,,,52,,,2007-11-09 05:00:00,MULTIPOINT (-79.38087 43.64146)
2,3,13444656,Steeles Ave W,Carpenter Rd,3,3,3,STEELES AVE W,CARPENTER RD,6003,...,North York,York Centre(6),,,,32,,,2007-11-09 05:00:00,MULTIPOINT (-79.44759 43.79201)
3,4,13444138,Steeles Ave W,Hilda Ave,4,4,4,STEELES AVE W,HILDA AVE,6004,...,North York,Willowdale(18),,,,32,,,2007-11-09 05:00:00,MULTIPOINT (-79.42927 43.79601)
4,5,13451893,Albion Rd,Silverstone Dr,5,5,5,ALBION RD,SILVERSTONE DR,6005,...,Etobicoke York,Etobicoke North(1),,,,23,,,2007-11-09 05:00:00,MULTIPOINT (-79.60009 43.74295)


## 2: Converting Dataframes into GeoDataframes
The following section converts each of the pandas dataframes created in section 1 into geodataframes for geospatial analysis.

### 2.1: Creating the GeoDataframes
This conversion is done by acquiring the longitude and latitude of each dataframe and creating point geometry columns using geopandas. Every single dataframe has a lon and lat with exception to the motor_collisions_df which has a geometry column (though this does not function as a geometry column), and as a result, some work has to be done to create the lon and latitude columns.

In [263]:
# The geometry column of the motor collisionss df contains a set of arrays embedded in each other. As a result, by
# repeatedly indexing the array, you can eventually extract the value itself (hence the 2 [0] followed by a [0] or [1]).
motor_collisions_df['lon'] = motor_collisions_df['coordinates'].apply(lambda x: x[0][0])
motor_collisions_df['lat'] = motor_collisions_df['coordinates'].apply(lambda x: x[0][1])
motor_collisions_df.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,type,coordinates,lon,lat
0,1,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,None,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]",-79.452490,43.656345
1,2,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,None,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]",-79.452490,43.656345
2,3,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,None,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]",-79.199786,43.801943
3,4,893184,2006,2006-01-01,236,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,...,None,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,MultiPoint,"[[-79.318797, 43.699595]]",-79.318797,43.699595
4,5,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,None,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]",-79.199786,43.801943


In [264]:
# We turn the df into a gdf so that it has geometry.
motor_collisions_gdf = gpd.GeoDataFrame(motor_collisions_df, 
                                          geometry = gpd.points_from_xy(motor_collisions_df['lon'], 
                                                                        motor_collisions_df['lat'])
                                       )
motor_collisions_gdf.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,type,coordinates,lon,lat,geometry
0,1,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]",-79.452490,43.656345,POINT (-79.45249 43.65635)
1,2,892658,2006,2006-03-11,852,BLOOR ST W,DUNDAS ST W,None,Major Arterial,Toronto and East York,...,88,High Park North,88,High Park North (88),D11,MultiPoint,"[[-79.45249, 43.656345]]",-79.452490,43.656345,POINT (-79.45249 43.65635)
2,3,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]",-79.199786,43.801943,POINT (-79.19979 43.80194)
3,4,893184,2006,2006-01-01,236,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,...,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,MultiPoint,"[[-79.318797, 43.699595]]",-79.318797,43.699595,POINT (-79.31880 43.69960)
4,5,892810,2006,2006-03-11,915,MORNINGSIDE AVE,SHEPPARD AVE E,None,Major Arterial,Scarborough,...,146,Malvern East,132,Malvern (132),D42,MultiPoint,"[[-79.199786, 43.801943]]",-79.199786,43.801943,POINT (-79.19979 43.80194)


In [265]:
# Similarly, converting the intersections and traffic dfs into gdfs so that they have geometry.
intersections_gdf = gpd.GeoDataFrame(intersections, 
                                          geometry = gpd.points_from_xy(intersections['lng'], 
                                                                        intersections['lat'])
                                       )

vol_00to09_gdf = gpd.GeoDataFrame(vol_00to09, 
                                          geometry = gpd.points_from_xy(vol_00to09['lng'], 
                                                                        vol_00to09['lat'])
                                       )
vol_10to19_gdf = gpd.GeoDataFrame(vol_10to19, 
                                          geometry = gpd.points_from_xy(vol_10to19['lng'], 
                                                                        vol_10to19['lat'])
                                       )
vol_20to29_gdf = gpd.GeoDataFrame(vol_20to29, 
                                          geometry = gpd.points_from_xy(vol_20to29['lng'], 
                                                                        vol_20to29['lat'])
                                       )
inter_att_gdf = gpd.GeoDataFrame(inter_att_df, 
                                    geometry = gpd.points_from_xy(inter_att_df['coordinates'].
                                                                apply(lambda x : x[0][0]), 
                                                                inter_att_df['coordinates'].
                                                                apply(lambda x : x[0][1])
                                                                 )
                                   )

### 2.2 Assigning a Coordinate Reference System (CRS) to each GeoDataframe
A coordinate reference system (CRS) shows how projected points correspond to real locations on Earth. Each GeoDataFrame's crs is checked using the .crs method.  

In [266]:
# Doing this check finds that there is no coordinate system assigned to any of the gdfs, so we can pick one to assign it.
print(motor_collisions_gdf.crs)
print(intersections_gdf.crs)
print(vol_00to09_gdf.crs)
print(vol_10to19_gdf.crs)
print(vol_20to29_gdf.crs)
print(inter_att_gdf.crs)


None
None
None
None
None
None


Doing this check finds that there are varying/no coordinate system assigned to the gdfs, so we can pick one to assign it. As we're working in a relatively small area (Toronto) and we'll be looking at areas and distances, we transform to a 2D projection crs. The City of Mississauga distributes datasets in UTM NAD83 Zone 17N (EPSG:26917) so we set the crs to this. 

In [267]:
#The crs of the motor_collisions, and intersection and traffic gdfs will be initialized to (EPSG:26917).
motor_collisions_gdf.crs = {'init': 'epsg:26917'}
intersections_gdf.crs = {'init': 'epsg:26917'}
vol_00to09_gdf.crs = {'init': 'epsg:26917'}
vol_10to19_gdf.crs = {'init': 'epsg:26917'}
vol_20to29_gdf.crs = {'init': 'epsg:26917'}
inter_att_gdf.crs = {'init': 'epsg:26917'}


## 3: Creating a Master Collision-Volume-Intersection Attribute GeoDataframe
The following section looks to create a master dataframe that merges the traffic count volumes data (stored in vol_00to09_gdf, vol_10to19_gdf, and vol_20to29_gdf) corresponding to each collision in the motor_collisions_gdf. The master will also include intersection characteristics from inter_att_gdf. This will allow each collision to be contexualized in terms of the traffic volumes and accident frequency at intersections.

### 3.1: Aggregating the Volume GeoDataframes
A summative traffic volume count geodataframe will be created to aggregate all the historical data from vol_20to29_gdf, vol_10to19_gdf, and vol_00to09_gdf.

In [268]:
# Combine all volume GeoDataFrames into one
volumes_gdf = pd.concat([vol_20to29_gdf, vol_10to19_gdf, vol_00to09_gdf], ignore_index=True)

# Print the combined GeoDataFrame
volumes_gdf.head()

,_id,count_id,count_date,location_id,location,lng,lat,centreline_type,centreline_id,px,...,wx_peds,nx_bike,sx_bike,ex_bike,wx_bike,nx_other,sx_other,ex_other,wx_other,geometry
0,1,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2.0,13462138.0,NaN,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.359 43.678)
1,2,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2.0,13462138.0,NaN,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.359 43.678)
2,3,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2.0,13462138.0,NaN,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.359 43.678)
3,4,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2.0,13462138.0,NaN,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.359 43.678)
4,5,39337,2020-01-08,13060,BROADVIEW AVE AT ERINDALE AVE,-79.358652,43.677521,2.0,13462138.0,NaN,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-79.359 43.678)


### 3.2: Filtering for the Target Analysis Years
This motor collisions analysis defined an analysis period of 11 years (2011-2022) of the available data such that sufficient data is included before, during, and after the COVID pandemic. Both the motor_collisions_gdf and the volumes_gdf will be filtered to only include data from 2011-2022.

In [269]:
# Keep the years 2011-2022
# Filter motor_collisions_gdf
motor_collisions_gdf = motor_collisions_gdf[(motor_collisions_gdf['YEAR'] >= 2011) & (motor_collisions_gdf['YEAR'] <= 2022)]

# Filter volumes_gdf
volumes_gdf = volumes_gdf[(volumes_gdf['count_date'] >= '2011-01-01') & (volumes_gdf['count_date'] <= '2022-12-31')]

### 3.3: Converting Date and Time Columns to Datetime Objects
In order to conduct temporal analysis on the two GeoDataframes and eventually merge them, the dates and times of the collisions and traffic volume counts need to be consistently formatted, this is achieved by turning them into datetime objects. 

In [270]:
# Create a copy of motor_collisions_gdf
motor_collisions_gdf_copy = motor_collisions_gdf.copy()

# Pad the TIME values with zeros and convert to a valid time format (HH:MM)
motor_collisions_gdf_copy['TIME'] = motor_collisions_gdf_copy['TIME'].apply(lambda x: x.zfill(4))
motor_collisions_gdf_copy['datetime'] = pd.to_datetime(motor_collisions_gdf_copy['DATE'] + ' ' + motor_collisions_gdf_copy['TIME'].str[:2] + ':' + motor_collisions_gdf_copy['TIME'].str[2:])

# Display the updated DataFrame
motor_collisions_gdf_copy[['DATE', 'TIME', 'datetime']].head()

,DATE,TIME,datetime
6000,2011-03-09,0726,2011-03-09 07:26:00
6001,2011-03-09,1145,2011-03-09 11:45:00
6002,2011-03-09,1145,2011-03-09 11:45:00
6003,2011-03-09,1145,2011-03-09 11:45:00
6004,2011-03-12,1917,2011-03-12 19:17:00


In [271]:
# Create a copy of volumes_gdf
volumes_gdf_copy = volumes_gdf.copy()

# Add the 'datetime_start' and 'datetime_end' columns
volumes_gdf_copy['datetime_start'] = pd.to_datetime(volumes_gdf_copy['time_start'])
volumes_gdf_copy['datetime_end'] = pd.to_datetime(volumes_gdf_copy['time_end'])

# Display the updated DataFrame
volumes_gdf_copy[['count_date', 'time_start', 'datetime_start', 'time_end', 'datetime_end']].head()

,count_date,time_start,datetime_start,time_end,datetime_end
0,2020-01-08,2020-01-08T07:30:00,2020-01-08 07:30:00,2020-01-08T07:45:00,2020-01-08 07:45:00
1,2020-01-08,2020-01-08T07:45:00,2020-01-08 07:45:00,2020-01-08T08:00:00,2020-01-08 08:00:00
2,2020-01-08,2020-01-08T08:00:00,2020-01-08 08:00:00,2020-01-08T08:15:00,2020-01-08 08:15:00
3,2020-01-08,2020-01-08T08:15:00,2020-01-08 08:15:00,2020-01-08T08:30:00,2020-01-08 08:30:00
4,2020-01-08,2020-01-08T08:30:00,2020-01-08 08:30:00,2020-01-08T08:45:00,2020-01-08 08:45:00


### 3.4: Creating the Daily and Hourly Volumes Dataframes
The volumes_gdf is comprised of data for a single day at a single location. A TMC typically includes data for a total of 8 non-continuous hours throughout the day, and the data is reported in a series of 15-minute intervals. Analysis for traffic flow is typically done on an hourly and daily basis. The goal is to merge the relevant hourly and daily volume data for each location and time a collision took place. To achieve this, a daily_volumes_gdf and hourly_volumes_gdf will be created to merge with motor_collisions_gdf.

In [272]:
# Specify the columns to sum
columns_to_sum = ['sb_cars_r', 'sb_cars_t', 'sb_cars_l', 'nb_cars_r',
                  'nb_cars_t', 'nb_cars_l', 'wb_cars_r', 'wb_cars_t', 'wb_cars_l',
                  'eb_cars_r', 'eb_cars_t', 'eb_cars_l', 'sb_truck_r', 'sb_truck_t',
                  'sb_truck_l', 'nb_truck_r', 'nb_truck_t', 'nb_truck_l', 'wb_truck_r',
                  'wb_truck_t', 'wb_truck_l', 'eb_truck_r', 'eb_truck_t', 'eb_truck_l',
                  'sb_bus_r', 'sb_bus_t', 'sb_bus_l', 'nb_bus_r', 'nb_bus_t', 'nb_bus_l',
                  'wb_bus_r', 'wb_bus_t', 'wb_bus_l', 'eb_bus_r', 'eb_bus_t', 'eb_bus_l',
                  'nx_peds', 'sx_peds', 'ex_peds', 'wx_peds', 'nx_bike', 'sx_bike',
                  'ex_bike', 'wx_bike', 'nx_other', 'sx_other', 'ex_other', 'wx_other']

# Group by 'count_date' and 'location', and sum the specified columns with 'daily_' prefix
daily_volumes_gdf = volumes_gdf_copy.groupby(['count_date', 'location', 'location_id', 'lng', 'lat', 'geometry'])[columns_to_sum].sum().reset_index()

# Convert 'count_date' to pandas datetime object
daily_volumes_gdf['count_date'] = pd.to_datetime(daily_volumes_gdf['count_date'])

# Rename columns with 'daily_' prefix
daily_volumes_gdf.columns = ['daily_' + col if col in columns_to_sum else col for col in daily_volumes_gdf.columns]

# Display the new DataFrame
daily_volumes_gdf.head()

,count_date,location,location_id,lng,lat,geometry,daily_sb_cars_r,daily_sb_cars_t,daily_sb_cars_l,daily_nb_cars_r,...,daily_ex_peds,daily_wx_peds,daily_nx_bike,daily_sx_bike,daily_ex_bike,daily_wx_bike,daily_nx_other,daily_sx_other,daily_ex_other,daily_wx_other
0,2011-01-04,ASHTONBEE ROAD AT HAKIMI AVE (PX 2069),33555,-79.292071,43.729972,POINT (-79.292 43.730),41.0,16.0,64.0,634.0,...,105.0,50.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
1,2011-01-04,O CONNOR DR AT SUNRISE AVE (PX 520),4414,-79.304468,43.718906,POINT (-79.304 43.719),494.0,4149.0,293.0,344.0,...,276.0,59.0,48.0,0.0,51.0,0.0,0.0,0.0,0.0,0.0
2,2011-01-04,PHARMACY AVE AT PRAIRIE DR & TEESDALE PL (PX 1...,4830,-79.285399,43.696998,POINT (-79.285 43.697),301.0,3080.0,10.0,43.0,...,166.0,270.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0
3,2011-01-05,EASTERN AVE AT KNOX AVE (PX 1265),5521,-79.325270,43.662516,POINT (-79.325 43.663),658.0,75.0,76.0,237.0,...,175.0,55.0,19.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0
4,2011-01-06,DENLOW BLVD AT LESLIE ST & SOUTHWELL DR (PX 737),5980,-79.358519,43.745983,POINT (-79.359 43.746),348.0,8023.0,57.0,113.0,...,43.0,136.0,0.0,9.0,0.0,5.0,0.0,0.0,0.0,0.0


In [273]:
# Specify the columns to sum
columns_to_sum = ['sb_cars_r', 'sb_cars_t', 'sb_cars_l', 'nb_cars_r',
                  'nb_cars_t', 'nb_cars_l', 'wb_cars_r', 'wb_cars_t', 'wb_cars_l',
                  'eb_cars_r', 'eb_cars_t', 'eb_cars_l', 'sb_truck_r', 'sb_truck_t',
                  'sb_truck_l', 'nb_truck_r', 'nb_truck_t', 'nb_truck_l', 'wb_truck_r',
                  'wb_truck_t', 'wb_truck_l', 'eb_truck_r', 'eb_truck_t', 'eb_truck_l',
                  'sb_bus_r', 'sb_bus_t', 'sb_bus_l', 'nb_bus_r', 'nb_bus_t', 'nb_bus_l',
                  'wb_bus_r', 'wb_bus_t', 'wb_bus_l', 'eb_bus_r', 'eb_bus_t', 'eb_bus_l',
                  'nx_peds', 'sx_peds', 'ex_peds', 'wx_peds', 'nx_bike', 'sx_bike',
                  'ex_bike', 'wx_bike', 'nx_other', 'sx_other', 'ex_other', 'wx_other']

# Round datetime objects to the nearest hour
volumes_gdf_copy['hourly_datetime'] = pd.to_datetime(volumes_gdf_copy['datetime_start']).dt.round('H')

# Group by 'hourly_datetime', 'location', and 'location_id', and sum the specified columns with 'hourly_' prefix
hourly_volumes_gdf = volumes_gdf_copy.groupby(['hourly_datetime', 'location', 'location_id', 'lng', 'lat', 'geometry'])[columns_to_sum].sum().reset_index()

# Rename columns with 'hourly_' prefix
hourly_volumes_gdf.columns = ['hourly_' + col if col in columns_to_sum else col for col in hourly_volumes_gdf.columns]

# Display the new DataFrame
hourly_volumes_gdf.head()

,hourly_datetime,location,location_id,lng,lat,geometry,hourly_sb_cars_r,hourly_sb_cars_t,hourly_sb_cars_l,hourly_nb_cars_r,...,hourly_ex_peds,hourly_wx_peds,hourly_nx_bike,hourly_sx_bike,hourly_ex_bike,hourly_wx_bike,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other
0,2011-01-04 08:00:00,ASHTONBEE ROAD AT HAKIMI AVE (PX 2069),33555,-79.292071,43.729972,POINT (-79.292 43.730),3.0,2.0,10.0,47.0,...,15.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2011-01-04 08:00:00,O CONNOR DR AT SUNRISE AVE (PX 520),4414,-79.304468,43.718906,POINT (-79.304 43.719),128.0,735.0,38.0,48.0,...,41.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2011-01-04 08:00:00,PHARMACY AVE AT PRAIRIE DR & TEESDALE PL (PX 1...,4830,-79.285399,43.696998,POINT (-79.285 43.697),26.0,542.0,3.0,20.0,...,4.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2011-01-04 09:00:00,ASHTONBEE ROAD AT HAKIMI AVE (PX 2069),33555,-79.292071,43.729972,POINT (-79.292 43.730),0.0,1.0,3.0,34.0,...,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2011-01-04 09:00:00,O CONNOR DR AT SUNRISE AVE (PX 520),4414,-79.304468,43.718906,POINT (-79.304 43.719),72.0,423.0,17.0,33.0,...,28.0,11.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


### 3.5: Finding the Closest Counts for each Collision
In order to join the motor_collisions_gdf with the hourly_volumes_gdf and daily_volumes_gdf, the coordinates of the count intersections (from intersections_gdf) and the collision intersection need to be matched up. In order to do this, the function find_closest_location() was created to find the coordinates of the closest count to the coordinates of the collision.

In [274]:
intersections_gdf_copy = intersections_gdf.copy()

# Create a KDTree for fast nearest-neighbor queries
kdtree = cKDTree(intersections_gdf_copy[['lat', 'lng']])

# Create a function to find the closest location
def find_closest_location(row):
    # Coordinates from motor_collisions_gdf
    motor_coords = (row['lat'], row['lon'])

    # Query the KDTree to find the index of the nearest neighbor
    _, closest_location_index = kdtree.query(motor_coords)

    try:
        # Get the name of the closest location
        closest_location = intersections_gdf_copy.loc[closest_location_index, 'location']
    except KeyError:
        # Handle the case where the index is not present
        closest_location = None

    return closest_location

# Apply the function to create the 'closest_count_location' column
motor_collisions_gdf_copy['closest_count_location'] = motor_collisions_gdf_copy.apply(find_closest_location, axis=1)

# Display the updated DataFrame
motor_collisions_gdf_copy.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,HOOD_140,NEIGHBOURHOOD_140,DIVISION,type,coordinates,lon,lat,geometry,datetime,closest_count_location
6000,6001,1237939,2011,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,78,Kensington-Chinatown (78),D14,MultiPoint,"[[-79.40769, 43.656445]]",-79.40769,43.656445,POINT (-79.408 43.656),2011-03-09 07:26:00,BATHURST ST AT COLLEGE ST (PX 300)
6001,6002,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,89,Runnymede-Bloor West Village (89),D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523)
6002,6003,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,89,Runnymede-Bloor West Village (89),D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523)
6003,6004,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,89,Runnymede-Bloor West Village (89),D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523)
6004,6005,1222866,2011,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,75,Church-Yonge Corridor (75),D51,MultiPoint,"[[-79.37789, 43.658245]]",-79.37789,43.658245,POINT (-79.378 43.658),2011-03-12 19:17:00,CHURCH ST AT GOULD ST (PX 993)


There are some closest_count_locations that do not match up with the intersections from the motor_collisions_gdf ('STREET1' and 'STREET2') because counts do not exist for these locations. Some instances of this mismatching results in a count from a neighbouring intersection being associated with the collision location, while other instances result in a completely distant count. Let's identify these problematic intersections in both cases.

In [275]:
# Indexing the first word of each street to get the root name
motor_collisions_gdf_copy['street1_first_word'] = motor_collisions_gdf_copy['STREET1'].str.split().str[0]
motor_collisions_gdf_copy['street2_first_word'] = motor_collisions_gdf_copy['STREET2'].str.split().str[0]

# Filter rows based on whether the root name exists within the closest_count_location
problematic_intersections_far_away = motor_collisions_gdf_copy[
    ~motor_collisions_gdf_copy.apply(
        lambda row: (
            row['closest_count_location'].lower().find(row['street1_first_word'].lower()) != -1 or
            row['closest_count_location'].lower().find(row['street2_first_word'].lower()) != -1
        ),
        axis=1
    )
]

# Display the problematic intersections far away DataFrame
problematic_intersections_far_away.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,DIVISION,type,coordinates,lon,lat,geometry,datetime,closest_count_location,street1_first_word,street2_first_word
6022,6023,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6023,6024,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6024,6025,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6039,6040,1224559,2011,2011-03-24,1520,WESTON RD,BELLEVUE Cres,None,Minor Arterial,Etobicoke York,...,D12,MultiPoint,"[[-79.51539, 43.699845]]",-79.51539,43.699845,POINT (-79.515 43.700),2011-03-24 15:20:00,LAWRENCE AVE AT SOUTH STATION ST,WESTON,BELLEVUE
6040,6041,1224559,2011,2011-03-24,1520,WESTON RD,BELLEVUE Cres,None,Minor Arterial,Etobicoke York,...,D12,MultiPoint,"[[-79.51539, 43.699845]]",-79.51539,43.699845,POINT (-79.515 43.700),2011-03-24 15:20:00,LAWRENCE AVE AT SOUTH STATION ST,WESTON,BELLEVUE


In [276]:
len(problematic_intersections_far_away) / len(motor_collisions_gdf_copy)

0.1235517897285146

12% of the collisions have a closest_count_location that is far away from its actual location. 

In [277]:
# Indexing the first word of each street to get the root name
motor_collisions_gdf_copy['street1_first_word'] = motor_collisions_gdf_copy['STREET1'].str.split().str[0]
motor_collisions_gdf_copy['street2_first_word'] = motor_collisions_gdf_copy['STREET2'].str.split().str[0]

# Filter rows based on whether the root name exists within the closest_count_location
problematic_intersections_all = motor_collisions_gdf_copy[
    ~motor_collisions_gdf_copy.apply(
        lambda row: (
            row['closest_count_location'].lower().find(row['street1_first_word'].lower()) != -1 and
            row['closest_count_location'].lower().find(row['street2_first_word'].lower()) != -1
        ),
        axis=1
    )
]

# Display the problematic intersections all DataFrame
problematic_intersections_all.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,DIVISION,type,coordinates,lon,lat,geometry,datetime,closest_count_location,street1_first_word,street2_first_word
6022,6023,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6023,6024,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6024,6025,1233411,2011,2011-03-19,1600,LESLIE st,HARRIET st,None,Local,Toronto and East York,...,D55,MultiPoint,"[[-79.33389, 43.671645]]",-79.33389,43.671645,POINT (-79.334 43.672),2011-03-19 16:00:00,BUSHELL AVE AT MYRTLE AVE,LESLIE,HARRIET
6028,6029,1213866,2011,2011-01-21,0519,DUFFERIN ST,NORTON AVE W,None,Major Arterial,Toronto and East York,...,D13,MultiPoint,"[[-79.44329, 43.678645]]",-79.44329,43.678645,POINT (-79.443 43.679),2011-01-21 05:19:00,DUFFERIN ST AT ST CLAIR AVE W,DUFFERIN,NORTON
6032,6033,1224587,2011,2011-03-20,2215,MARKHAM RD,DUNELM ST,None,Major Arterial,Scarborough,...,D43,MultiPoint,"[[-79.22069, 43.747145]]",-79.22069,43.747145,POINT (-79.221 43.747),2011-03-20 22:15:00,COUGAR CRT AT LUELLA ST & MARKHAM RD (PX 990),MARKHAM,DUNELM


In [278]:
len(problematic_intersections_all) / len(motor_collisions_gdf_copy)

0.29249524468269067

30% of the collisions do not have an exact count. 

#### Solution: Drop only the collisions with mismatched counts that are far away
To preserve 26% amount of the collisions data, only the mismatched locations that have counts that are far away will be dropped. This would mean that all the collisions in the master dataframe have counts that are from the collision location OR similar counts to what would be expected at the location of the collision. 

In [279]:
# motor_collisions_gdf_copy without all the far away problematic intersections
master_df = motor_collisions_gdf_copy[~motor_collisions_gdf_copy.index.isin(problematic_intersections_far_away.index)].copy()

# Display the master_df DataFrame
master_df.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,DIVISION,type,coordinates,lon,lat,geometry,datetime,closest_count_location,street1_first_word,street2_first_word
6000,6001,1237939,2011,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,D14,MultiPoint,"[[-79.40769, 43.656445]]",-79.40769,43.656445,POINT (-79.408 43.656),2011-03-09 07:26:00,BATHURST ST AT COLLEGE ST (PX 300),COLLEGE,BATHURST
6001,6002,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523),JANE,ST
6002,6003,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523),JANE,ST
6003,6004,1232522,2011,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,D11,MultiPoint,"[[-79.48969, 43.661945]]",-79.48969,43.661945,POINT (-79.490 43.662),2011-03-09 11:45:00,JANE ST AT ST JOHNS RD (PX 523),JANE,ST
6004,6005,1222866,2011,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,D51,MultiPoint,"[[-79.37789, 43.658245]]",-79.37789,43.658245,POINT (-79.378 43.658),2011-03-12 19:17:00,CHURCH ST AT GOULD ST (PX 993),CHURCH,GOULD


### 3.6: Combining the Daily and Hourly Volumes Dataframes with the Motor Collisions Dataframe

First merging the daily volumes to each collision

In [280]:
def merge_daily_volumes(row):
    # Extract location and date information
    location = row['closest_count_location']
    date = row['datetime']  # Use 'datetime' column directly
    
    # Filter daily_volumes_gdf for the exact location match
    location_match = daily_volumes_gdf[daily_volumes_gdf['location'].str.lower() == location.lower()]
    
    if location_match.empty:
        return pd.Series()  # Return an empty Series if there are no matches
    
    # Find the closest date
    location_match['date_diff'] = abs((location_match['count_date'] - pd.to_datetime(date)).dt.total_seconds())
    
    if location_match.empty:
        return pd.Series()  # Return an empty Series if there are no matches after date filtering
    
    closest_date_index = location_match['date_diff'].idxmin()
    
    # Get the corresponding row from daily_volumes_gdf
    closest_row = location_match.loc[closest_date_index]
    
    # Drop the temporary column
    closest_row = closest_row.drop('date_diff')
    
    # Merge the rows
    merged_row = pd.concat([row, closest_row], ignore_index=True)
    
    return merged_row

# Apply the function to create the merged DataFrame
master_df_merged = master_df.apply(merge_daily_volumes, axis=1)

# Display the updated DataFrame
master_df_merged.head()


,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
6000,6001.0,1237939,2011.0,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,1923.0,1190.0,253.0,234.0,797.0,1647.0,0.0,0.0,0.0,0.0
6001,6002.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,210.0,117.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0
6002,6003.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,210.0,117.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0
6003,6004.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,210.0,117.0,9.0,17.0,20.0,10.0,0.0,0.0,0.0,0.0
6004,6005.0,1222866,2011.0,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,1540.0,934.0,567.0,387.0,129.0,156.0,0.0,0.0,0.0,0.0


Then merging the hourly volumes to each collision

In [281]:
def merge_hourly_volumes(row):
    # Extract location and date information
    location = row[58]
    date_time = row[57]
    
    # Check for NaN values
    if pd.isna(location) or pd.isna(date_time):
        return pd.Series()  # Return an empty Series if there are NaN values
    
    # Filter hourly_volumes_gdf for the exact location match
    location_match = hourly_volumes_gdf[hourly_volumes_gdf['location'].str.lower() == location.lower()]
    
    if location_match.empty:
        return pd.Series()  # Return an empty Series if there are no matches
    
    # Find the closest datetime
    location_match['datetime_diff'] = abs((location_match['hourly_datetime'] - pd.to_datetime(date_time)).dt.total_seconds())
    
    if location_match.empty:
        return pd.Series()  # Return an empty Series if there are no matches after datetime filtering
    
    closest_datetime_index = location_match['datetime_diff'].idxmin()
    
    # Get the corresponding row from hourly_volumes_gdf
    closest_row = location_match.loc[closest_datetime_index]
    
    # Drop the temporary column
    closest_row = closest_row.drop('datetime_diff')
    
    # Merge the rows
    merged_row = pd.concat([row, closest_row])
    
    # Reset the index to default (0, 1, 2, ...)
    merged_row = merged_row.reset_index(drop=True)
    
    return merged_row

# Apply the function to create the merged DataFrame for hourly volumes
master_df_hourly_merged = master_df_merged.apply(merge_hourly_volumes, axis=1)


# Display the updated DataFrame
master_df_hourly_merged.head()

,0,1,2,3,4,5,6,7,8,9,...,159,160,161,162,163,164,165,166,167,168
6000,6001.0,1237939,2011.0,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,208.0,160.0,50.0,2.0,43.0,312.0,0.0,0.0,0.0,0.0
6001,6002.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6002,6003.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6003,6004.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6004,6005.0,1222866,2011.0,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,222.0,97.0,106.0,27.0,31.0,9.0,0.0,0.0,0.0,0.0


In [282]:
# Set the columns to the desired column names
master_df_hourly_merged.columns = ['_id', 'ACCNUM', 'YEAR', 'DATE', 'TIME', 'STREET1', 'STREET2', 'OFFSET',
       'ROAD_CLASS', 'DISTRICT', 'WARDNUM', 'LOCCOORD', 'ACCLOC', 'TRAFFCTL',
       'VISIBILITY', 'LIGHT', 'RDSFCOND', 'ACCLASS', 'IMPACTYPE', 'INVTYPE',
       'INVAGE', 'INJURY', 'FATAL_NO', 'INITDIR', 'VEHTYPE', 'MANOEUVER',
       'DRIVACT', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE',
       'CYCACT', 'CYCCOND', 'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE',
       'MOTORCYCLE', 'TRUCK', 'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER',
       'SPEEDING', 'AG_DRIV', 'REDLIGHT', 'ALCOHOL', 'DISABILITY', 'HOOD_158',
       'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140', 'DIVISION',
       'type', 'collision_coordinates', 'collision_lon', 'collision_lat', 'collision_geometry', 'collision_datetime',
       'closest_count_location', 'street1_first_word', 'street2_first_word', 'count_date', 'count_location', 'count_location_id', 
        'count_lng', 'count_lat', 'count_geometry',
       'daily_sb_cars_r', 'daily_sb_cars_t', 'daily_sb_cars_l',
       'daily_nb_cars_r', 'daily_nb_cars_t', 'daily_nb_cars_l',
       'daily_wb_cars_r', 'daily_wb_cars_t', 'daily_wb_cars_l',
       'daily_eb_cars_r', 'daily_eb_cars_t', 'daily_eb_cars_l',
       'daily_sb_truck_r', 'daily_sb_truck_t', 'daily_sb_truck_l',
       'daily_nb_truck_r', 'daily_nb_truck_t', 'daily_nb_truck_l',
       'daily_wb_truck_r', 'daily_wb_truck_t', 'daily_wb_truck_l',
       'daily_eb_truck_r', 'daily_eb_truck_t', 'daily_eb_truck_l',
       'daily_sb_bus_r', 'daily_sb_bus_t', 'daily_sb_bus_l', 'daily_nb_bus_r',
       'daily_nb_bus_t', 'daily_nb_bus_l', 'daily_wb_bus_r', 'daily_wb_bus_t',
       'daily_wb_bus_l', 'daily_eb_bus_r', 'daily_eb_bus_t', 'daily_eb_bus_l',
       'daily_nx_peds', 'daily_sx_peds', 'daily_ex_peds', 'daily_wx_peds',
       'daily_nx_bike', 'daily_sx_bike', 'daily_ex_bike', 'daily_wx_bike',
       'daily_nx_other', 'daily_sx_other', 'daily_ex_other', 'daily_wx_other',
        'count_datetime', 'hourly_location', 'hourly_location_id', 'hourly_lng', 'hourly_lat', 'hourly_geometry',
       'hourly_sb_cars_r', 'hourly_sb_cars_t', 'hourly_sb_cars_l',
       'hourly_nb_cars_r', 'hourly_nb_cars_t', 'hourly_nb_cars_l',
       'hourly_wb_cars_r', 'hourly_wb_cars_t', 'hourly_wb_cars_l',
       'hourly_eb_cars_r', 'hourly_eb_cars_t', 'hourly_eb_cars_l',
       'hourly_sb_truck_r', 'hourly_sb_truck_t', 'hourly_sb_truck_l',
       'hourly_nb_truck_r', 'hourly_nb_truck_t', 'hourly_nb_truck_l',
       'hourly_wb_truck_r', 'hourly_wb_truck_t', 'hourly_wb_truck_l',
       'hourly_eb_truck_r', 'hourly_eb_truck_t', 'hourly_eb_truck_l',
       'hourly_sb_bus_r', 'hourly_sb_bus_t', 'hourly_sb_bus_l',
       'hourly_nb_bus_r', 'hourly_nb_bus_t', 'hourly_nb_bus_l',
       'hourly_wb_bus_r', 'hourly_wb_bus_t', 'hourly_wb_bus_l',
       'hourly_eb_bus_r', 'hourly_eb_bus_t', 'hourly_eb_bus_l',
       'hourly_nx_peds', 'hourly_sx_peds', 'hourly_ex_peds', 'hourly_wx_peds',
       'hourly_nx_bike', 'hourly_sx_bike', 'hourly_ex_bike', 'hourly_wx_bike',
       'hourly_nx_other', 'hourly_sx_other', 'hourly_ex_other',
       'hourly_wx_other'                   
                           ]


In [283]:
master_df_hourly_merged.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_ex_peds,hourly_wx_peds,hourly_nx_bike,hourly_sx_bike,hourly_ex_bike,hourly_wx_bike,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other
6000,6001.0,1237939,2011.0,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,208.0,160.0,50.0,2.0,43.0,312.0,0.0,0.0,0.0,0.0
6001,6002.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6002,6003.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6003,6004.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6004,6005.0,1222866,2011.0,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,222.0,97.0,106.0,27.0,31.0,9.0,0.0,0.0,0.0,0.0


### 3.7: Mild Data Cleaning

In [284]:
# Removing null rows
master_df_hourly_merged = master_df_hourly_merged.dropna(how='all')
master_df_hourly_merged.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_ex_peds,hourly_wx_peds,hourly_nx_bike,hourly_sx_bike,hourly_ex_bike,hourly_wx_bike,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other
6000,6001.0,1237939,2011.0,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,208.0,160.0,50.0,2.0,43.0,312.0,0.0,0.0,0.0,0.0
6001,6002.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6002,6003.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6003,6004.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
6004,6005.0,1222866,2011.0,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,222.0,97.0,106.0,27.0,31.0,9.0,0.0,0.0,0.0,0.0


In [285]:
# Resetting the row indices
master_df_hourly_merged.reset_index(drop=True, inplace=True)
master_df_hourly_merged.head()

,_id,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_ex_peds,hourly_wx_peds,hourly_nx_bike,hourly_sx_bike,hourly_ex_bike,hourly_wx_bike,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other
0,6001.0,1237939,2011.0,2011-03-09,0726,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,208.0,160.0,50.0,2.0,43.0,312.0,0.0,0.0,0.0,0.0
1,6002.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
2,6003.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
3,6004.0,1232522,2011.0,2011-03-09,1145,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
4,6005.0,1222866,2011.0,2011-03-12,1917,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,222.0,97.0,106.0,27.0,31.0,9.0,0.0,0.0,0.0,0.0


In [286]:
# Drop the temporary columns made to merge the dataframes
columns_to_drop = ['_id', 'type', 'collision_lon', 'collision_lat', 'collision_coordinates', 'closest_count_location', 'street1_first_word', 'street2_first_word', 
                  'count_date',  'count_lng', 'count_lat', 'hourly_location', 'hourly_location_id', 'hourly_lng', 'hourly_lat', 'hourly_geometry']

master_df_hourly_merged.drop(columns=columns_to_drop, inplace=True)

In [287]:
#Reordering the columns
desired_order = ['ACCNUM', 'YEAR', 'DATE', 'TIME', 'collision_datetime', 'STREET1', 'STREET2', 'OFFSET',
       'ROAD_CLASS', 'DISTRICT', 'WARDNUM', 'LOCCOORD', 'ACCLOC', 'TRAFFCTL',
       'VISIBILITY', 'LIGHT', 'RDSFCOND', 'ACCLASS', 'IMPACTYPE', 'INVTYPE',
       'INVAGE', 'INJURY', 'FATAL_NO', 'INITDIR', 'VEHTYPE', 'MANOEUVER',
       'DRIVACT', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE',
       'CYCACT', 'CYCCOND', 'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE',
       'MOTORCYCLE', 'TRUCK', 'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER',
       'SPEEDING', 'AG_DRIV', 'REDLIGHT', 'ALCOHOL', 'DISABILITY', 'HOOD_158',
       'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140', 'DIVISION',
     'collision_geometry', 
          'count_datetime', 'count_location', 'count_location_id',
        'count_geometry',
       'daily_sb_cars_r', 'daily_sb_cars_t', 'daily_sb_cars_l',
       'daily_nb_cars_r', 'daily_nb_cars_t', 'daily_nb_cars_l',
       'daily_wb_cars_r', 'daily_wb_cars_t', 'daily_wb_cars_l',
       'daily_eb_cars_r', 'daily_eb_cars_t', 'daily_eb_cars_l',
       'daily_sb_truck_r', 'daily_sb_truck_t', 'daily_sb_truck_l',
       'daily_nb_truck_r', 'daily_nb_truck_t', 'daily_nb_truck_l',
       'daily_wb_truck_r', 'daily_wb_truck_t', 'daily_wb_truck_l',
       'daily_eb_truck_r', 'daily_eb_truck_t', 'daily_eb_truck_l',
       'daily_sb_bus_r', 'daily_sb_bus_t', 'daily_sb_bus_l', 'daily_nb_bus_r',
       'daily_nb_bus_t', 'daily_nb_bus_l', 'daily_wb_bus_r', 'daily_wb_bus_t',
       'daily_wb_bus_l', 'daily_eb_bus_r', 'daily_eb_bus_t', 'daily_eb_bus_l',
       'daily_nx_peds', 'daily_sx_peds', 'daily_ex_peds', 'daily_wx_peds',
       'daily_nx_bike', 'daily_sx_bike', 'daily_ex_bike', 'daily_wx_bike',
       'daily_nx_other', 'daily_sx_other', 'daily_ex_other', 'daily_wx_other',
       
       'hourly_sb_cars_r', 'hourly_sb_cars_t', 'hourly_sb_cars_l',
       'hourly_nb_cars_r', 'hourly_nb_cars_t', 'hourly_nb_cars_l',
       'hourly_wb_cars_r', 'hourly_wb_cars_t', 'hourly_wb_cars_l',
       'hourly_eb_cars_r', 'hourly_eb_cars_t', 'hourly_eb_cars_l',
       'hourly_sb_truck_r', 'hourly_sb_truck_t', 'hourly_sb_truck_l',
       'hourly_nb_truck_r', 'hourly_nb_truck_t', 'hourly_nb_truck_l',
       'hourly_wb_truck_r', 'hourly_wb_truck_t', 'hourly_wb_truck_l',
       'hourly_eb_truck_r', 'hourly_eb_truck_t', 'hourly_eb_truck_l',
       'hourly_sb_bus_r', 'hourly_sb_bus_t', 'hourly_sb_bus_l',
       'hourly_nb_bus_r', 'hourly_nb_bus_t', 'hourly_nb_bus_l',
       'hourly_wb_bus_r', 'hourly_wb_bus_t', 'hourly_wb_bus_l',
       'hourly_eb_bus_r', 'hourly_eb_bus_t', 'hourly_eb_bus_l',
       'hourly_nx_peds', 'hourly_sx_peds', 'hourly_ex_peds', 'hourly_wx_peds',
       'hourly_nx_bike', 'hourly_sx_bike', 'hourly_ex_bike', 'hourly_wx_bike',
       'hourly_nx_other', 'hourly_sx_other', 'hourly_ex_other',
       'hourly_wx_other'                   
                           ]
master_df_hourly_merged_reordered = master_df_hourly_merged[desired_order]
master_df_hourly_merged_reordered.head()

,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_ex_peds,hourly_wx_peds,hourly_nx_bike,hourly_sx_bike,hourly_ex_bike,hourly_wx_bike,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other
0,1237939,2011.0,2011-03-09,0726,2011-03-09 07:26:00,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,208.0,160.0,50.0,2.0,43.0,312.0,0.0,0.0,0.0,0.0
1,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
2,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
3,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,59.0,36.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
4,1222866,2011.0,2011-03-12,1917,2011-03-12 19:17:00,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,222.0,97.0,106.0,27.0,31.0,9.0,0.0,0.0,0.0,0.0


### 3.7: Merging Intersection Characteristics

In [288]:
def find_closest_intersection(row, inter_att_gdf, kdtree):
    collision_point = row['collision_geometry']
    
    # Query the KDTree to find the index of the nearest point
    _, idx = kdtree.query([collision_point.x, collision_point.y])
    
    # Find the corresponding row in inter_att_gdf
    nearest_intersection_row = inter_att_gdf.iloc[idx]
    
    # Append the information from the nearest intersection to the original row
    for col in inter_att_gdf.columns:
        row[col] = nearest_intersection_row[col]
    
    return row

# Extract coordinates from inter_att_gdf for KDTree construction
intersection_coords = inter_att_gdf['geometry'].apply(lambda geom: (geom.x, geom.y)).tolist()

# Build a KDTree using intersection coordinates
kdtree = KDTree(intersection_coords)

# Apply the function to each row in master_col_vol
master_col_vol = master_col_vol.apply(lambda row: find_closest_intersection(row, inter_att_gdf, kdtree), axis=1)

In [289]:
master_col_vol.head()

,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,ELEVATION_UNIT,HEIGHT_RESTRICTION,HEIGHT_RESTRICTION_UNIT,STATE,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,type,coordinates,geometry
0,1237939,2011.0,2011-03-09,0726,2011-03-09 07:26:00,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,,0.0,,8,200000,-1,17969,MultiPoint,"[[-79.4077229391202, 43.6564819597087]]",POINT (-79.4077229391202 43.6564819597087)
1,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,,0.0,,8,200000,-1,4482,MultiPoint,"[[-79.4896709148133, 43.6619933565023]]",POINT (-79.4896709148133 43.6619933565023)
2,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,,0.0,,8,200000,-1,4482,MultiPoint,"[[-79.4896709148133, 43.6619933565023]]",POINT (-79.4896709148133 43.6619933565023)
3,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,,0.0,,8,200000,-1,4482,MultiPoint,"[[-79.4896709148133, 43.6619933565023]]",POINT (-79.4896709148133 43.6619933565023)
4,1222866,2011.0,2011-03-12,1917,2011-03-12 19:17:00,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,,0.0,,8,200000,-1,39551,MultiPoint,"[[-79.3778551199192, 43.6582110922312]]",POINT (-79.3778551199192 43.6582110922312)


In [290]:
columns_to_drop2 = ['_id', 'INTERSECTION_ID', 'DATE_EFFECTIVE', 
                   'DATE_EXPIRY', 'ELEVATION_ID', 'ELEVATION_FEATURE_CODE',
                    'ELEVATION_FEATURE_CODE_DESC', 'ELEVATION_LEVEL', 
                    'ELEVATION','ELEVATION_UNIT','HEIGHT_RESTRICTION', 
                    'HEIGHT_RESTRICTION_UNIT', 'STATE', 
                    'TRANS_ID_CREATE', 'TRANS_ID_EXPIRE', 
                    'OBJECTID', 'type', 'coordinates']

                   
master_col_vol.drop(columns=columns_to_drop2, inplace = True)
master_col_vol.head()

,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_sx_other,hourly_ex_other,hourly_wx_other,INTERSECTION_DESC,CLASSIFICATION,CLASSIFICATION_DESC,NUMBER_OF_ELEVATIONS,inter_att_geometry,traffic_calming,geometry
0,1237939,2011.0,2011-03-09,0726,2011-03-09 07:26:00,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,0.0,0.0,0.0,College St / Bathurst St,MJRSL,Major-Single Level,1,POINT (-79.4077229391202 43.6564819597087),0,POINT (-79.4077229391202 43.6564819597087)
1,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,0.0,0.0,0.0,Jane St / St John's Rd,MNRSL,Minor-Single Level,1,POINT (-79.4896709148133 43.6619933565023),0,POINT (-79.4896709148133 43.6619933565023)
2,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,0.0,0.0,0.0,Jane St / St John's Rd,MNRSL,Minor-Single Level,1,POINT (-79.4896709148133 43.6619933565023),0,POINT (-79.4896709148133 43.6619933565023)
3,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,0.0,0.0,0.0,Jane St / St John's Rd,MNRSL,Minor-Single Level,1,POINT (-79.4896709148133 43.6619933565023),0,POINT (-79.4896709148133 43.6619933565023)
4,1222866,2011.0,2011-03-12,1917,2011-03-12 19:17:00,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,0.0,0.0,0.0,Gould St / Church St,MNRSL,Minor-Single Level,1,POINT (-79.3778551199192 43.6582110922312),0,POINT (-79.3778551199192 43.6582110922312)


In [291]:
# Rename 'geometry' column to 'inter_att_geometry' in master_col_vol
master_col_vol = master_col_vol.rename(columns={'geometry': 'inter_att_geometry'})

### 3.7: Merging Intersection Safety


In [292]:
traffic_calming_gdf['geometry']

0      LINESTRING (325456.886 4842936.976, 325501.009...
1      LINESTRING (309818.658 4839638.193, 309785.082...
2      LINESTRING (313963.651 4839945.340, 314325.237...
3      LINESTRING (313413.250 4838443.321, 313363.575...
4      LINESTRING (309897.291 4836672.254, 309882.452...
                             ...                        
860    LINESTRING (319295.562 4838253.021, 319344.791...
861    LINESTRING (319805.857 4837857.973, 319893.957...
862    LINESTRING (322441.851 4837135.278, 322443.622...
863    LINESTRING (319670.395 4838058.587, 319667.271...
864    LINESTRING (303197.773 4840635.639, 303231.519...
Name: geometry, Length: 865, dtype: geometry

In [294]:
traffic_calming_gdf.crs

<Projected CRS: EPSG:2952>
Name: NAD83(CSRS) / MTM zone 10
Axis Info [cartesian]:
- E(X)[east]: Easting (metre)
- N(Y)[north]: Northing (metre)
Area of Use:
- name: Canada - Quebec west of 78°W; Canada - Ontario - between 79°30'W and 78°W in area to north of 47°N; between 80°15'W and 78°W in area between 46°N and 47°N; between 81°W and 78°W in area south of 46°N.
- bounds: (-81.0, 42.26, -78.0, 62.45)
Coordinate Operation:
- name: MTM zone 10
- method: Transverse Mercator
Datum: NAD83 Canadian Spatial Reference System
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [293]:
master_col_vol[['collision_geometry']]

,collision_geometry
0,POINT (-79.40769 43.656445)
1,POINT (-79.48969 43.661945)
2,POINT (-79.48969 43.661945)
3,POINT (-79.48969 43.661945)
4,POINT (-79.37789 43.658245)
...,...
7967,POINT (-79.436534 43.696864)
7968,POINT (-79.436534 43.696864)
7969,POINT (-79.436534 43.696864)
7970,POINT (-79.421268 43.662856)


In [297]:
# Create a GeoDataFrame from your DataFrame
master_col_vol = gpd.GeoDataFrame(master_col_vol, geometry='collision_geometry')

# Optional: Set the coordinate reference system (CRS) if needed
master_col_vol.crs = "EPSG:4326"  # WGS 84

In [303]:
# Function to create a Folium map with a MarkerCluster and title
def create_map(gdf, name, color, title):
    # Create a Folium map centered around Toronto
    toronto_map = folium.Map(location=[43.70, -79.42], zoom_start=11)

    # Convert GeoDataFrame to GeoJSON format
    gdf_json = gdf.to_crs(epsg='4326').to_json()

    # Create a MarkerCluster for the GeoJSON layer
    marker_cluster = MarkerCluster(name=name).add_to(toronto_map)

    # Add GeoJSON layer to the map with specified color
    folium.GeoJson(gdf_json, style_function=lambda x: {'color': color}).add_to(marker_cluster)

    # Add title to the map
    title_html = f'<h3>{title}</h3>'
    folium.Html(title_html, script=True).add_to(toronto_map)
    
    # Display the map
    folium.LayerControl().add_to(toronto_map)
    return toronto_map

# Create and display maps for each GeoDataFrame with titles
map_traffic_calming = create_map(traffic_calming_gdf, 'Traffic Calming', 'blue', 'Traffic Calming Map')
map_traffic_calming

In [307]:
# Create a map centered around Toronto
toronto_map = folium.Map(location=[43.70, -79.42], zoom_start=11)

# Create a FastMarkerCluster with a list of points
points = list(zip(master_col_vol['collision_geometry'].y, master_col_vol['collision_geometry'].x))
marker_cluster = FastMarkerCluster(points).add_to(toronto_map)

# Display the map
toronto_map

In [308]:
# Function to create a Folium map with a MarkerCluster and title
def create_map(gdf, name, color, title, map_instance=None):
    if map_instance is None:
        # Create a Folium map centered around Toronto
        toronto_map = folium.Map(location=[43.70, -79.42], zoom_start=11)
    else:
        toronto_map = map_instance

    # Convert GeoDataFrame to GeoJSON format
    gdf_json = gdf.to_crs(epsg='4326').to_json()

    # Create a MarkerCluster for the GeoJSON layer
    marker_cluster = MarkerCluster(name=name).add_to(toronto_map)

    # Add GeoJSON layer to the map with specified color
    folium.GeoJson(gdf_json, style_function=lambda x: {'color': color}).add_to(marker_cluster)

    # Add title to the map
    title_html = f'<h3>{title}</h3>'
    folium.Html(title_html, script=True).add_to(toronto_map)

    return toronto_map

# Create a map for traffic calming
map_traffic_calming = create_map(traffic_calming_gdf, 'Traffic Calming', 'blue', 'Traffic Calming Map')

# Create a FastMarkerCluster with a list of points for collisions
points = list(zip(master_col_vol['collision_geometry'].y, master_col_vol['collision_geometry'].x))
marker_cluster = FastMarkerCluster(points).add_to(map_traffic_calming)

# Display the map
map_traffic_calming


In [326]:
def add_traffic_calming_column(master_col_vol, traffic_calming_gdf, buffer_distance=10):
    # Drop rows with invalid geometries
    traffic_calming_gdf = traffic_calming_gdf[traffic_calming_gdf['geometry'].notna()]

    # Reproject the GeoDataFrames to a projected CRS (e.g., UTM)
    master_col_vol = master_col_vol.to_crs(traffic_calming_gdf.crs)

    # Buffer the collision points by the specified distance
    buffered_collision_points = master_col_vol['collision_geometry'].buffer(buffer_distance)

    # Create a unary union of the buffered collision points
    union_buffer = unary_union(buffered_collision_points)

    # Use GeoPandas spatial methods to check if the union buffer intersects with traffic calming linestrings
    master_col_vol['traffic_calming'] = master_col_vol.apply(
        lambda row: 1 if row['collision_geometry'].buffer(buffer_distance).intersects(traffic_calming_gdf['geometry'].unary_union) else 0,
        axis=1
    )

    return master_col_vol

# Example usage
master_col_vol = add_traffic_calming_column(master_col_vol, traffic_calming_gdf)
master_col_vol[['traffic_calming']]

,traffic_calming
0,0
1,0
2,0
3,0
4,0
...,...
7967,0
7968,0
7969,0
7970,0


In [327]:
# Check the distribution of values in 'traffic_calming' column
traffic_calming_distribution = master_col_vol['traffic_calming'].value_counts()

# Display the result
print(traffic_calming_distribution)

0    7411
1     561
Name: traffic_calming, dtype: int64


In [330]:
master_col_vol = master_col_vol.loc[:, ~master_col_vol.columns.duplicated(keep='first')]

In [331]:
master_col_vol.head()

,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_nx_other,hourly_sx_other,hourly_ex_other,hourly_wx_other,INTERSECTION_DESC,CLASSIFICATION,CLASSIFICATION_DESC,NUMBER_OF_ELEVATIONS,inter_att_geometry,traffic_calming
0,1237939,2011.0,2011-03-09,0726,2011-03-09 07:26:00,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,0.0,0.0,0.0,0.0,College St / Bathurst St,MJRSL,Major-Single Level,1,POINT (-79.4077229391202 43.6564819597087),0
1,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,0.0,0.0,0.0,0.0,Jane St / St John's Rd,MNRSL,Minor-Single Level,1,POINT (-79.4896709148133 43.6619933565023),0
2,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,0.0,0.0,0.0,0.0,Jane St / St John's Rd,MNRSL,Minor-Single Level,1,POINT (-79.4896709148133 43.6619933565023),0
3,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,0.0,0.0,0.0,0.0,Jane St / St John's Rd,MNRSL,Minor-Single Level,1,POINT (-79.4896709148133 43.6619933565023),0
4,1222866,2011.0,2011-03-12,1917,2011-03-12 19:17:00,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,0.0,0.0,0.0,0.0,Gould St / Church St,MNRSL,Minor-Single Level,1,POINT (-79.3778551199192 43.6582110922312),0


In [353]:
speed_enforcement_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [345]:
map_speed_enforcement = create_map(speed_enforcement_gdf, 'Speed Enforcement', 'green', 'Speed Enforcement Map')

# Display the map
map_speed_enforcement

In [343]:
map_red_light_cameras = create_map(red_light_cameras_gdf, 'Red Light Cameras', 'red', 'Red Light Cameras Map')
map_red_light_cameras

In [348]:
def add_cameras_enforcement_columns(master_col_vol, red_light_cameras_gdf, speed_enforcement_gdf, buffer_distance=20):
    # Drop rows with invalid geometries
    red_light_cameras_gdf = red_light_cameras_gdf[red_light_cameras_gdf['geometry'].notna()]
    speed_enforcement_gdf = speed_enforcement_gdf[speed_enforcement_gdf['geometry'].notna()]

    # Buffer the collision points by the specified distance
    buffered_collision_points = master_col_vol['collision_geometry'].buffer(buffer_distance)

    # Create a unary union of the buffered collision points
    union_buffer = unary_union(buffered_collision_points)

    # Use GeoPandas spatial methods to check intersections with red_light_cameras_gdf
    master_col_vol['red_light_cam'] = master_col_vol.apply(
        lambda row: 1 if row['collision_geometry'].buffer(buffer_distance).within(red_light_cameras_gdf['geometry'].unary_union) else 0,
        axis=1
    )

    # Use GeoPandas spatial methods to check intersections with speed_enforcement_gdf
    master_col_vol['speed_enforcement'] = master_col_vol.apply(
        lambda row: 1 if row['collision_geometry'].buffer(buffer_distance).within(speed_enforcement_gdf['geometry'].unary_union) else 0,
        axis=1
    )

    return master_col_vol


master_col_vol = add_cameras_enforcement_columns(master_col_vol, red_light_cameras_gdf, speed_enforcement_gdf)
master_col_vol[['red_light_cam', 'speed_enforcement']]


,red_light_cam,speed_enforcement
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
7967,0,0
7968,0,0
7969,0,0
7970,0,0


In [366]:
def add_cameras_enforcement_columns(master_col_vol, red_light_cameras_gdf, speed_enforcement_gdf):
    # Drop rows with invalid geometries
    red_light_cameras_gdf = red_light_cameras_gdf[red_light_cameras_gdf['geometry'].notna()]
    speed_enforcement_gdf = speed_enforcement_gdf[speed_enforcement_gdf['geometry'].notna()]

    # Reproject to a suitable projected CRS (e.g., UTM) before buffering
    red_light_cameras_gdf = red_light_cameras_gdf.to_crs(master_col_vol.crs)
    speed_enforcement_gdf = speed_enforcement_gdf.to_crs(master_col_vol.crs)

    # Buffer the multipoints by the specified distance
    buffered_red_light_cameras = red_light_cameras_gdf['geometry'].buffer(10)
    buffered_speed_enforcement = speed_enforcement_gdf['geometry'].buffer(20)

    # Check if each collision point is within the buffered multipoints
    master_col_vol['red_light_cam'] = master_col_vol['collision_geometry'].apply(
        lambda point: 1 if any(point.within(buffer) for buffer in buffered_red_light_cameras) else 0
    )

    master_col_vol['speed_enforcement'] = master_col_vol['collision_geometry'].apply(
        lambda point: 1 if any(point.intersects(buffer) for buffer in buffered_speed_enforcement) else 0
    )

    return master_col_vol

# Example usage
master_col_vol = add_cameras_enforcement_columns(master_col_vol, red_light_cameras_gdf, speed_enforcement_gdf)
master_col_vol[['red_light_cam', 'speed_enforcement']]



,red_light_cam,speed_enforcement
0,1,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
7967,0,0
7968,0,0
7969,0,0
7970,0,0


In [369]:
master_col_vol.head()

,ACCNUM,YEAR,DATE,TIME,collision_datetime,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,hourly_ex_other,hourly_wx_other,INTERSECTION_DESC,CLASSIFICATION,CLASSIFICATION_DESC,NUMBER_OF_ELEVATIONS,inter_att_geometry,traffic_calming,red_light_cam,speed_enforcement
0,1237939,2011.0,2011-03-09,0726,2011-03-09 07:26:00,COLLEGE Stre,BATHURST Stre,None,Major Arterial,Toronto and East York,...,0.0,0.0,College St / Bathurst St,MJRSL,Major-Single Level,1,POINT (-79.4077229391202 43.6564819597087),0,1,0
1,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,0.0,0.0,Jane St / St John's Rd,MNRSL,Minor-Single Level,1,POINT (-79.4896709148133 43.6619933565023),0,0,0
2,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,0.0,0.0,Jane St / St John's Rd,MNRSL,Minor-Single Level,1,POINT (-79.4896709148133 43.6619933565023),0,0,0
3,1232522,2011.0,2011-03-09,1145,2011-03-09 11:45:00,JANE ST,ST JOHNS RD,None,Major Arterial,Toronto and East York,...,0.0,0.0,Jane St / St John's Rd,MNRSL,Minor-Single Level,1,POINT (-79.4896709148133 43.6619933565023),0,0,0
4,1222866,2011.0,2011-03-12,1917,2011-03-12 19:17:00,CHURCH ST,GOULD ST,None,Minor Arterial,Toronto and East York,...,0.0,0.0,Gould St / Church St,MNRSL,Minor-Single Level,1,POINT (-79.3778551199192 43.6582110922312),0,0,0


In [370]:
# Rename the DataFrame
master_gdf = master_col_vol

# Save the DataFrame to a CSV file in the same folder as the code
master_gdf.to_csv("master_gdf.csv", index=False)